First we create LabelOps instance to work with specific Label Studio project, referring it by ID.

1. Get your **Access Token** on *Account & Settings* page.

2. Create project with the following configuration:

```xml
<View>
  <Text name="text" value="$text"/>
    <Choices name="label" toName="text" choice="single" showInLine="true">
      <Choice value="Positive"/>
      <Choice value="Negative"/>
    </Choices>
</View>
```

3. Upload tasks (note current limitation of <1000 tasks per project)

In [17]:
import json
from label_studio_sdk.labelops import LabelOps

labelops = LabelOps(project_id=123, api_key='Your-Access-Token')

Next step is to initialize and index current project tasks. This process could take a while, therefore we can check project initialization status to be in *FINISHED* state.

Only tasks' data will be processed, annotations / predictions could be built after. Annotations' data is going to be used to assess LabelOps results

In [15]:
labelops.initialize_project()

{'loaded': [312]}

In [18]:
labelops.check_project_status()

b'FINISHED'

Then we can declare our LabelOps. Each LabelOp consists of 
- `"name"`: arbitrary LabelOp identifier
- `"label"`: actual label that is going to be assigned
- `"pattern"`: matching pattern

Two types of text-based matching patterns are used here:

1. **Regex** - it takes task data by specified query key (`"text"`) and apply regular expression to match
2. **Entities** - having a list of entities of different types (coming from dependency parser and named entity recognizer), it tries to find the dependency pattern that follows the item order in the list. In the example bellow, it searches for `"VERB"` ([part-of-speech tag](https://spacy.io/usage/linguistic-features#pos-tagging))  pointed to some generic `"PERSON"` named entity, that matches a pattern like "Mr. Brown says...".

In [53]:
# ['CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART']
params = [{
    "name": "Product Positive Feedback",
    "label": "Positive",
    "pattern": [{
        "type": "Entities",
        "query": [
            {"type": "ner", "label": "PRODUCT"}
        ]
    }, {
        "type": "Regex",
        "query": {
            "text": ".*I\s+would\s+recommend.*"
        }
    }]
}, {
    "name": "Overall Negative Feedback",
    "label": "Negative",
    "pattern": [{
        "type": "Regex",
        "query": {
            "text": ".*(junk|crap).*"
        }
    }]
}]

In [54]:
result = labelops.apply(params)

After applying declared LabelOps to existed project, we can evaluate various performance statistics over LabelOps like
- **coverage** - how well LabelOp cover the dataset
- **conflict** - how many conflicts current LabelOp produces with other LabelOps
- **accuracy** - empirical accuracy based on existing annotations. You can always annotate more to get more ground truth samples to get more accurate estimates.

In [55]:
import pandas as pd
pd.DataFrame.from_records(result["stats"])

,name,coverage,conflict,accuracy
0,Product Positive Feedback,0.083333,0.0,0.0
1,Overall Negative Feedback,0.916667,0.0,0.0


We can also check how well our LabelOps perform on existing tasks when being aggregated...

In [56]:
pd.DataFrame.from_records(result["aggregated"])

,task,text,label
0,6679359,Absolutel junk.,Negative
1,6679481,The worst piece of crap ever along with the Ve...,Negative
2,6679556,BT50 battery junk!.,Negative
3,6679590,Pretty piece of junk.,Negative
4,6679598,Defective crap.,Negative
5,6679691,"What possesed me to get this junk, I have no i...",Negative
6,6679710,the worst phone from samsung...crap..... this ...,Negative
7,6679749,Utter crap.. Sound quality is TERRIBLE.,Negative
8,6680015,"Saggy, floppy piece of junk.",Negative
9,6680092,What a piece of junk.. I lose more calls on th...,Negative


...and finally commit them as "Predictions" to the target project, by making version control via `model_version` parameter.

In [57]:
labelops.commit(result, model_version="MyFirstTry")

{'predictions_count': 12}